In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from html_table_parser import parser_functions as parser
import pandas as pd
import time

In [21]:
# 현재 페이지의 표를 데이터 프레임 형으로 반환하는 메소드

def makeTable(mon, year):
    html = driver.page_source #지금 현 상태의 page source불러오기
    soup = bs(html,'html.parser')
    table = soup.find(id='mySheet1')#sheet 영역 불러오기
    p=parser.make2d(table)
    df0=pd.DataFrame(p)
    
    # 열 데이터 중복제거
    for i in range(0,33,2):
        df0=df0.drop(i, axis=1)
        
    df0 = df0.drop(31, axis=1) # 끝에 쓸모 없는거 제거
    df0 = df0.drop(5, axis=1) # 중간 쓸모 없는거 제거

    #18년 데이터 제거
    for i in range(9, 21,2):
        df0 = df0.drop(i, axis=1)

    # 행 데이터 불필요 제거
    for i in range(0,len(df0)):
        if (df0[3][i]==None or df0[3][i]=='' or df0[3][i]=='코드'):
            df0=df0.drop(i)
    # 색인 설정
    df0.columns=['순번','코드','품목명','수출중량', '수출증감률','수입중량', '수입증감률', '차이']
    
    #수량 값 float 변환
    df0['수출중량'] = df0['수출중량'].str.replace(',','').astype(float)
    df0['수입중량'] = df0['수입중량'].str.replace(',','').astype(float)
    

    # 물동량 데이터 생성
    #df0[year+'_'+mon+'월 '+'물동량'] = df0[mon+'수출중량']+df0[mon+'수입중량']
    # 인덱스 재설정
    df0=df0.rename(columns={'수입중량':year+'_'+mon+'월 수입중량'})
    df0=df0.rename(columns={'수출중량':year+'_'+mon+'월 수출중량'})
    # 번호 재설정
    #df0=df0.reset_index(drop=True)
    df0 = df0.drop('품목명', axis=1)
    df0 = df0.drop('순번', axis=1)
    #df0 = df0.drop(str(mon)+'수출중량', axis=1)
    df0 = df0.drop('수출증감률', axis=1)
    #df0 = df0.drop(str(mon)+'수입중량', axis=1)
    df0 = df0.drop('수입증감률', axis=1)
    df0 = df0.drop('차이', axis=1)
    
    df0 = df0.set_index('코드')
    

    return df0

In [28]:
df_final=pd.DataFrame()

#크롤링
chromedriver_path = 'C:/Users/종찬/chromedriver_win32/chromedriver.exe'

driver = webdriver.Chrome(chromedriver_path)
driver.implicitly_wait(3)
driver.get('http://stat.kita.net/stat/kts/port/PortItemImpExpList.screen#none')

#국가 전체로 바꾸기
e = driver.find_element_by_xpath('//*[@id="s_ctr_code"]')
e.clear()
e.send_keys("")

#HS 코드 2단위
hsk = driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[1]/div/select[2]')
hsk.send_keys('2')

 #중량 기준 설정
weight = driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[2]/div[3]/select')
weight.send_keys('중량')

#당월 선택
d = driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[2]/div[4]/select')
d.send_keys('당월')

#단위 선택
ton = driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[2]/div[5]/select')
ton.send_keys('t')

#정렬 기준
code = driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[2]/div[6]/select')
code.send_keys('코드')

#오름차순
asc = driver.find_element_by_xpath('//*[@id="s_sort_val_asc"]')
asc.send_keys(' ')




for code in ['50','51','52','53','54','55','56','57','58','59','60','61','62','63']:
    
    df_result = pd.DataFrame()
    s_code = code
    
    #시작 코드
    c = driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[1]/label[3]/input')
    c.clear()
    c.send_keys(s_code)
                
    for year in [3, 2]:
        str_y = str(2021-year)
        year_xpath = '//*[@id="contents"]/div[2]/form/fieldset/div[2]/div[1]/select/option['+str(year)+']'
        print(str_y+'년 ')
        
        for month in range(1,13):
            str_m = str(month)
            
            print(str_m+'월 진행 중...==== '+s_code+' ====')
            
            
            #년월 수정
            year = driver.find_element_by_xpath(year_xpath)
            year.click()
            
            month = driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[2]/div[2]/select/option['+str_m+']')
            month.click()

            #데이터 수량 조절
            data_num = driver.find_element_by_xpath('//*[@id="listCount"]')
            data_num.send_keys('1')
            
            time.sleep(1)
            
            #조회 버튼 클릭
            driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[3]/a/img').click()
            
            time.sleep(3)
            
            df_crowl=makeTable(str_m, str_y)
            
            if year==3 and month==1: df_result = df_crowl
            else: df_result = pd.concat([df_result,df_crowl],axis=1)
                
    if s_code == '50': df_final = df_result
    else: df_final = pd.concat([df_final, df_result])
    print(df_final)

2018년 
1월 진행 중...==== 50 ====
2월 진행 중...==== 50 ====
3월 진행 중...==== 50 ====
4월 진행 중...==== 50 ====
5월 진행 중...==== 50 ====
6월 진행 중...==== 50 ====
7월 진행 중...==== 50 ====
8월 진행 중...==== 50 ====
9월 진행 중...==== 50 ====
10월 진행 중...==== 50 ====
11월 진행 중...==== 50 ====
12월 진행 중...==== 50 ====
2019년 
1월 진행 중...==== 50 ====
2월 진행 중...==== 50 ====
3월 진행 중...==== 50 ====
4월 진행 중...==== 50 ====
5월 진행 중...==== 50 ====
6월 진행 중...==== 50 ====
7월 진행 중...==== 50 ====
8월 진행 중...==== 50 ====
9월 진행 중...==== 50 ====
10월 진행 중...==== 50 ====
11월 진행 중...==== 50 ====
12월 진행 중...==== 50 ====
    2018_1월 수출중량  2018_1월 수입중량  2018_2월 수출중량  2018_2월 수입중량  2018_3월 수출중량  \
코드                                                                         
50           1.0          26.0           0.0          15.0           1.0   

    2018_3월 수입중량  2018_4월 수출중량  2018_4월 수입중량  2018_5월 수출중량  2018_5월 수입중량  ...  \
코드                                                                        ...   
50          22.0           0.0       

8월 진행 중...==== 54 ====
9월 진행 중...==== 54 ====
10월 진행 중...==== 54 ====
11월 진행 중...==== 54 ====
12월 진행 중...==== 54 ====
2019년 
1월 진행 중...==== 54 ====
2월 진행 중...==== 54 ====
3월 진행 중...==== 54 ====
4월 진행 중...==== 54 ====
5월 진행 중...==== 54 ====
6월 진행 중...==== 54 ====
7월 진행 중...==== 54 ====
8월 진행 중...==== 54 ====
9월 진행 중...==== 54 ====
10월 진행 중...==== 54 ====
11월 진행 중...==== 54 ====
12월 진행 중...==== 54 ====
    2018_1월 수출중량  2018_1월 수입중량  2018_2월 수출중량  2018_2월 수입중량  2018_3월 수출중량  \
코드                                                                         
50           1.0          26.0           0.0          15.0           1.0   
51          56.0          41.0          64.0           9.0         110.0   
52         475.0        6980.0         325.0        7536.0         508.0   
53          36.0        1861.0          24.0        1353.0          44.0   
54        2245.0        4947.0        1776.0        4066.0        2444.0   

    2018_3월 수입중량  2018_4월 수출중량  2018_4월 수입중량  2018_5월 수출중량  201

2월 진행 중...==== 57 ====
3월 진행 중...==== 57 ====
4월 진행 중...==== 57 ====
5월 진행 중...==== 57 ====
6월 진행 중...==== 57 ====
7월 진행 중...==== 57 ====
8월 진행 중...==== 57 ====
9월 진행 중...==== 57 ====
10월 진행 중...==== 57 ====
11월 진행 중...==== 57 ====
12월 진행 중...==== 57 ====
2019년 
1월 진행 중...==== 57 ====
2월 진행 중...==== 57 ====
3월 진행 중...==== 57 ====
4월 진행 중...==== 57 ====
5월 진행 중...==== 57 ====
6월 진행 중...==== 57 ====
7월 진행 중...==== 57 ====
8월 진행 중...==== 57 ====
9월 진행 중...==== 57 ====
10월 진행 중...==== 57 ====
11월 진행 중...==== 57 ====
12월 진행 중...==== 57 ====
    2018_1월 수출중량  2018_1월 수입중량  2018_2월 수출중량  2018_2월 수입중량  2018_3월 수출중량  \
코드                                                                         
50           1.0          26.0           0.0          15.0           1.0   
51          56.0          41.0          64.0           9.0         110.0   
52         475.0        6980.0         325.0        7536.0         508.0   
53          36.0        1861.0          24.0        1353.0          44.0   
54

8월 진행 중...==== 59 ====
9월 진행 중...==== 59 ====
10월 진행 중...==== 59 ====
11월 진행 중...==== 59 ====
12월 진행 중...==== 59 ====
    2018_1월 수출중량  2018_1월 수입중량  2018_2월 수출중량  2018_2월 수입중량  2018_3월 수출중량  \
코드                                                                         
50           1.0          26.0           0.0          15.0           1.0   
51          56.0          41.0          64.0           9.0         110.0   
52         475.0        6980.0         325.0        7536.0         508.0   
53          36.0        1861.0          24.0        1353.0          44.0   
54        2245.0        4947.0        1776.0        4066.0        2444.0   
55        2256.0        8541.0        1909.0        6077.0        2329.0   
56         392.0        2690.0         251.0        2292.0         425.0   
57         509.0        1162.0         372.0         781.0         350.0   
58         208.0         437.0         162.0         230.0         235.0   
59         576.0        1744.0         421.0  

2월 진행 중...==== 61 ====
3월 진행 중...==== 61 ====
4월 진행 중...==== 61 ====
5월 진행 중...==== 61 ====
6월 진행 중...==== 61 ====
7월 진행 중...==== 61 ====
8월 진행 중...==== 61 ====
9월 진행 중...==== 61 ====
10월 진행 중...==== 61 ====
11월 진행 중...==== 61 ====
12월 진행 중...==== 61 ====
2019년 
1월 진행 중...==== 61 ====
2월 진행 중...==== 61 ====
3월 진행 중...==== 61 ====
4월 진행 중...==== 61 ====
5월 진행 중...==== 61 ====
6월 진행 중...==== 61 ====
7월 진행 중...==== 61 ====
8월 진행 중...==== 61 ====
9월 진행 중...==== 61 ====
10월 진행 중...==== 61 ====
11월 진행 중...==== 61 ====
12월 진행 중...==== 61 ====
    2018_1월 수출중량  2018_1월 수입중량  2018_2월 수출중량  2018_2월 수입중량  2018_3월 수출중량  \
코드                                                                         
50           1.0          26.0           0.0          15.0           1.0   
51          56.0          41.0          64.0           9.0         110.0   
52         475.0        6980.0         325.0        7536.0         508.0   
53          36.0        1861.0          24.0        1353.0          44.0   
54

2월 진행 중...==== 63 ====
3월 진행 중...==== 63 ====
4월 진행 중...==== 63 ====
5월 진행 중...==== 63 ====
6월 진행 중...==== 63 ====
7월 진행 중...==== 63 ====
8월 진행 중...==== 63 ====
9월 진행 중...==== 63 ====
10월 진행 중...==== 63 ====
11월 진행 중...==== 63 ====
12월 진행 중...==== 63 ====
2019년 
1월 진행 중...==== 63 ====
2월 진행 중...==== 63 ====
3월 진행 중...==== 63 ====
4월 진행 중...==== 63 ====
5월 진행 중...==== 63 ====
6월 진행 중...==== 63 ====
7월 진행 중...==== 63 ====
8월 진행 중...==== 63 ====
9월 진행 중...==== 63 ====
10월 진행 중...==== 63 ====
11월 진행 중...==== 63 ====
12월 진행 중...==== 63 ====
    2018_1월 수출중량  2018_1월 수입중량  2018_2월 수출중량  2018_2월 수입중량  2018_3월 수출중량  \
코드                                                                         
50           1.0          26.0           0.0          15.0           1.0   
51          56.0          41.0          64.0           9.0         110.0   
52         475.0        6980.0         325.0        7536.0         508.0   
53          36.0        1861.0          24.0        1353.0          44.0   
54

In [29]:
df_final

,2018_1월 수출중량,2018_1월 수입중량,2018_2월 수출중량,2018_2월 수입중량,2018_3월 수출중량,2018_3월 수입중량,2018_4월 수출중량,2018_4월 수입중량,2018_5월 수출중량,2018_5월 수입중량,...,2019_8월 수출중량,2019_8월 수입중량,2019_9월 수출중량,2019_9월 수입중량,2019_10월 수출중량,2019_10월 수입중량,2019_11월 수출중량,2019_11월 수입중량,2019_12월 수출중량,2019_12월 수입중량
코드,,,,,,,,,,,,,,,,,,,,,
50,1.0,26.0,0.0,15.0,1.0,22.0,0.0,23.0,1.0,21.0,...,0.0,18.0,9.0,21.0,0.0,26.0,0.0,20.0,0.0,21.0
51,56.0,41.0,64.0,9.0,110.0,12.0,207.0,23.0,234.0,62.0,...,253.0,64.0,128.0,104.0,113.0,94.0,118.0,56.0,40.0,40.0
52,475.0,6980.0,325.0,7536.0,508.0,9273.0,536.0,7951.0,751.0,7320.0,...,515.0,5825.0,465.0,6221.0,672.0,6860.0,675.0,7197.0,706.0,7563.0
53,36.0,1861.0,24.0,1353.0,44.0,1713.0,28.0,2447.0,41.0,2825.0,...,61.0,763.0,71.0,504.0,259.0,736.0,58.0,835.0,121.0,1024.0
54,2245.0,4947.0,1776.0,4066.0,2444.0,4945.0,2381.0,4427.0,3029.0,5709.0,...,2474.0,5011.0,2422.0,5114.0,2746.0,6338.0,2977.0,5916.0,2989.0,5639.0
55,2256.0,8541.0,1909.0,6077.0,2329.0,7333.0,2518.0,7755.0,2806.0,7098.0,...,2504.0,6622.0,2275.0,6891.0,2936.0,7930.0,2749.0,7779.0,2654.0,6731.0
56,392.0,2690.0,251.0,2292.0,425.0,2901.0,457.0,3243.0,436.0,3747.0,...,279.0,3252.0,261.0,3355.0,385.0,3708.0,403.0,3410.0,330.0,3188.0
57,509.0,1162.0,372.0,781.0,350.0,1036.0,456.0,1352.0,333.0,1684.0,...,124.0,1825.0,134.0,1945.0,211.0,2110.0,111.0,1911.0,46.0,2076.0
58,208.0,437.0,162.0,230.0,235.0,263.0,277.0,335.0,265.0,368.0,...,224.0,501.0,304.0,575.0,266.0,585.0,268.0,540.0,335.0,368.0


In [33]:
df_final.to_excel('방직용.xlsx')

In [34]:
df_final=pd.DataFrame()

#크롤링
chromedriver_path = 'C:/Users/종찬/chromedriver_win32/chromedriver.exe'

driver = webdriver.Chrome(chromedriver_path)
driver.implicitly_wait(3)
driver.get('http://stat.kita.net/stat/kts/port/PortItemImpExpList.screen#none')

#국가 전체로 바꾸기
e = driver.find_element_by_xpath('//*[@id="s_ctr_code"]')
e.clear()
e.send_keys("")

#HS 코드 2단위
hsk = driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[1]/div/select[2]')
hsk.send_keys('4')

 #중량 기준 설정
weight = driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[2]/div[3]/select')
weight.send_keys('중량')

#당월 선택
d = driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[2]/div[4]/select')
d.send_keys('당월')

#단위 선택
ton = driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[2]/div[5]/select')
ton.send_keys('t')

#정렬 기준
code = driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[2]/div[6]/select')
code.send_keys('코드')

#오름차순
asc = driver.find_element_by_xpath('//*[@id="s_sort_val_asc"]')
asc.send_keys(' ')




for code in ['8516','8544','8504','8528', '8501', '8507', '8518', '8529',
              '8541', '8517', '8509', '8536', '8545', '8548', '8503', '8505']:
    
    df_result = pd.DataFrame()
    s_code = code
    
    #시작 코드
    c = driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[1]/label[3]/input')
    c.clear()
    c.send_keys(s_code)
                
    for year in [3, 2]:
        
        str_y = str(2021-year)
        year_xpath = '//*[@id="contents"]/div[2]/form/fieldset/div[2]/div[1]/select/option['+str(year)+']'
        print(str_y+'년 ')
        
        for month in range(1,13):

    #['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']:
            str_m = str(month)
            
            print(str_m+'월 진행 중...==== '+s_code+' ====')
            
            
            #년월 수정
            year = driver.find_element_by_xpath(year_xpath)
            year.click()
            
            month = driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[2]/div[2]/select/option['+str_m+']')
            month.click()

            #데이터 수량 조절
            data_num = driver.find_element_by_xpath('//*[@id="listCount"]')
            data_num.send_keys('1')
            
            time.sleep(1)
            
            #조회 버튼 클릭
            driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[3]/a/img').click()
            
            time.sleep(3)
            
            df_crowl=makeTable(str_m, str_y)
            
            if year==3 and month==1: df_result = df_crowl
            else: df_result = pd.concat([df_result,df_crowl],axis=1)
                
    if s_code == '8516': df_final = df_result
    else: df_final = pd.concat([df_final, df_result])
    print(df_final)
driver.close()

2018년 
1월 진행 중...==== 8516 ====
2월 진행 중...==== 8516 ====
3월 진행 중...==== 8516 ====
4월 진행 중...==== 8516 ====
5월 진행 중...==== 8516 ====
6월 진행 중...==== 8516 ====
7월 진행 중...==== 8516 ====
8월 진행 중...==== 8516 ====
9월 진행 중...==== 8516 ====
10월 진행 중...==== 8516 ====
11월 진행 중...==== 8516 ====
12월 진행 중...==== 8516 ====
2019년 
1월 진행 중...==== 8516 ====
2월 진행 중...==== 8516 ====
3월 진행 중...==== 8516 ====
4월 진행 중...==== 8516 ====
5월 진행 중...==== 8516 ====
6월 진행 중...==== 8516 ====
7월 진행 중...==== 8516 ====
8월 진행 중...==== 8516 ====
9월 진행 중...==== 8516 ====
10월 진행 중...==== 8516 ====
11월 진행 중...==== 8516 ====
12월 진행 중...==== 8516 ====
      2018_1월 수출중량  2018_1월 수입중량  2018_2월 수출중량  2018_2월 수입중량  2018_3월 수출중량  \
코드                                                                           
8516         226.0        6315.0         166.0        5483.0         308.0   

      2018_3월 수입중량  2018_4월 수출중량  2018_4월 수입중량  2018_5월 수출중량  2018_5월 수입중량  \
코드                                                                 

2월 진행 중...==== 8501 ====
3월 진행 중...==== 8501 ====
4월 진행 중...==== 8501 ====
5월 진행 중...==== 8501 ====
6월 진행 중...==== 8501 ====
7월 진행 중...==== 8501 ====
8월 진행 중...==== 8501 ====
9월 진행 중...==== 8501 ====
10월 진행 중...==== 8501 ====
11월 진행 중...==== 8501 ====
12월 진행 중...==== 8501 ====
2019년 
1월 진행 중...==== 8501 ====
2월 진행 중...==== 8501 ====
3월 진행 중...==== 8501 ====
4월 진행 중...==== 8501 ====
5월 진행 중...==== 8501 ====
6월 진행 중...==== 8501 ====
7월 진행 중...==== 8501 ====
8월 진행 중...==== 8501 ====
9월 진행 중...==== 8501 ====
10월 진행 중...==== 8501 ====
11월 진행 중...==== 8501 ====
12월 진행 중...==== 8501 ====
      2018_1월 수출중량  2018_1월 수입중량  2018_2월 수출중량  2018_2월 수입중량  2018_3월 수출중량  \
코드                                                                           
8516         226.0        6315.0         166.0        5483.0         308.0   
8544        2983.0        6671.0        2609.0        5212.0        2896.0   
8504        1377.0        3196.0        1281.0        2613.0        1497.0   
8528         516.0    

2월 진행 중...==== 8529 ====
3월 진행 중...==== 8529 ====
4월 진행 중...==== 8529 ====
5월 진행 중...==== 8529 ====
6월 진행 중...==== 8529 ====
7월 진행 중...==== 8529 ====
8월 진행 중...==== 8529 ====
9월 진행 중...==== 8529 ====
10월 진행 중...==== 8529 ====
11월 진행 중...==== 8529 ====
12월 진행 중...==== 8529 ====
2019년 
1월 진행 중...==== 8529 ====
2월 진행 중...==== 8529 ====
3월 진행 중...==== 8529 ====
4월 진행 중...==== 8529 ====
5월 진행 중...==== 8529 ====
6월 진행 중...==== 8529 ====
7월 진행 중...==== 8529 ====
8월 진행 중...==== 8529 ====
9월 진행 중...==== 8529 ====
10월 진행 중...==== 8529 ====
11월 진행 중...==== 8529 ====
12월 진행 중...==== 8529 ====
      2018_1월 수출중량  2018_1월 수입중량  2018_2월 수출중량  2018_2월 수입중량  2018_3월 수출중량  \
코드                                                                           
8516         226.0        6315.0         166.0        5483.0         308.0   
8544        2983.0        6671.0        2609.0        5212.0        2896.0   
8504        1377.0        3196.0        1281.0        2613.0        1497.0   
8528         516.0    

2월 진행 중...==== 8517 ====
3월 진행 중...==== 8517 ====
4월 진행 중...==== 8517 ====
5월 진행 중...==== 8517 ====
6월 진행 중...==== 8517 ====
7월 진행 중...==== 8517 ====
8월 진행 중...==== 8517 ====
9월 진행 중...==== 8517 ====
10월 진행 중...==== 8517 ====
11월 진행 중...==== 8517 ====
12월 진행 중...==== 8517 ====
2019년 
1월 진행 중...==== 8517 ====
2월 진행 중...==== 8517 ====
3월 진행 중...==== 8517 ====
4월 진행 중...==== 8517 ====
5월 진행 중...==== 8517 ====
6월 진행 중...==== 8517 ====
7월 진행 중...==== 8517 ====
8월 진행 중...==== 8517 ====
9월 진행 중...==== 8517 ====
10월 진행 중...==== 8517 ====
11월 진행 중...==== 8517 ====
12월 진행 중...==== 8517 ====
      2018_1월 수출중량  2018_1월 수입중량  2018_2월 수출중량  2018_2월 수입중량  2018_3월 수출중량  \
코드                                                                           
8516         226.0        6315.0         166.0        5483.0         308.0   
8544        2983.0        6671.0        2609.0        5212.0        2896.0   
8504        1377.0        3196.0        1281.0        2613.0        1497.0   
8528         516.0    

2월 진행 중...==== 8536 ====
3월 진행 중...==== 8536 ====
4월 진행 중...==== 8536 ====
5월 진행 중...==== 8536 ====
6월 진행 중...==== 8536 ====
7월 진행 중...==== 8536 ====
8월 진행 중...==== 8536 ====
9월 진행 중...==== 8536 ====
10월 진행 중...==== 8536 ====
11월 진행 중...==== 8536 ====
12월 진행 중...==== 8536 ====
2019년 
1월 진행 중...==== 8536 ====
2월 진행 중...==== 8536 ====
3월 진행 중...==== 8536 ====
4월 진행 중...==== 8536 ====
5월 진행 중...==== 8536 ====
6월 진행 중...==== 8536 ====
7월 진행 중...==== 8536 ====
8월 진행 중...==== 8536 ====
9월 진행 중...==== 8536 ====
10월 진행 중...==== 8536 ====
11월 진행 중...==== 8536 ====
12월 진행 중...==== 8536 ====
      2018_1월 수출중량  2018_1월 수입중량  2018_2월 수출중량  2018_2월 수입중량  2018_3월 수출중량  \
코드                                                                           
8516         226.0        6315.0         166.0        5483.0         308.0   
8544        2983.0        6671.0        2609.0        5212.0        2896.0   
8504        1377.0        3196.0        1281.0        2613.0        1497.0   
8528         516.0    

2018년 
1월 진행 중...==== 8548 ====
2월 진행 중...==== 8548 ====
3월 진행 중...==== 8548 ====
4월 진행 중...==== 8548 ====
5월 진행 중...==== 8548 ====
6월 진행 중...==== 8548 ====
7월 진행 중...==== 8548 ====
8월 진행 중...==== 8548 ====
9월 진행 중...==== 8548 ====
10월 진행 중...==== 8548 ====
11월 진행 중...==== 8548 ====
12월 진행 중...==== 8548 ====
2019년 
1월 진행 중...==== 8548 ====
2월 진행 중...==== 8548 ====
3월 진행 중...==== 8548 ====
4월 진행 중...==== 8548 ====
5월 진행 중...==== 8548 ====
6월 진행 중...==== 8548 ====
7월 진행 중...==== 8548 ====
8월 진행 중...==== 8548 ====
9월 진행 중...==== 8548 ====
10월 진행 중...==== 8548 ====
11월 진행 중...==== 8548 ====
12월 진행 중...==== 8548 ====
      2018_1월 수출중량  2018_1월 수입중량  2018_2월 수출중량  2018_2월 수입중량  2018_3월 수출중량  \
코드                                                                           
8516         226.0        6315.0         166.0        5483.0         308.0   
8544        2983.0        6671.0        2609.0        5212.0        2896.0   
8504        1377.0        3196.0        1281.0        2613.0        

2월 진행 중...==== 8505 ====
3월 진행 중...==== 8505 ====
4월 진행 중...==== 8505 ====
5월 진행 중...==== 8505 ====
6월 진행 중...==== 8505 ====
7월 진행 중...==== 8505 ====
8월 진행 중...==== 8505 ====
9월 진행 중...==== 8505 ====
10월 진행 중...==== 8505 ====
11월 진행 중...==== 8505 ====
12월 진행 중...==== 8505 ====
2019년 
1월 진행 중...==== 8505 ====
2월 진행 중...==== 8505 ====
3월 진행 중...==== 8505 ====
4월 진행 중...==== 8505 ====
5월 진행 중...==== 8505 ====
6월 진행 중...==== 8505 ====
7월 진행 중...==== 8505 ====
8월 진행 중...==== 8505 ====
9월 진행 중...==== 8505 ====
10월 진행 중...==== 8505 ====
11월 진행 중...==== 8505 ====
12월 진행 중...==== 8505 ====
      2018_1월 수출중량  2018_1월 수입중량  2018_2월 수출중량  2018_2월 수입중량  2018_3월 수출중량  \
코드                                                                           
8516         226.0        6315.0         166.0        5483.0         308.0   
8544        2983.0        6671.0        2609.0        5212.0        2896.0   
8504        1377.0        3196.0        1281.0        2613.0        1497.0   
8528         516.0    

In [35]:
df_final

,2018_1월 수출중량,2018_1월 수입중량,2018_2월 수출중량,2018_2월 수입중량,2018_3월 수출중량,2018_3월 수입중량,2018_4월 수출중량,2018_4월 수입중량,2018_5월 수출중량,2018_5월 수입중량,...,2019_8월 수출중량,2019_8월 수입중량,2019_9월 수출중량,2019_9월 수입중량,2019_10월 수출중량,2019_10월 수입중량,2019_11월 수출중량,2019_11월 수입중량,2019_12월 수출중량,2019_12월 수입중량
코드,,,,,,,,,,,,,,,,,,,,,
8516,226.0,6315.0,166.0,5483.0,308.0,4901.0,304.0,5035.0,301.0,4972.0,...,260.0,4115.0,336.0,4958.0,380.0,7748.0,390.0,8896.0,295.0,7675.0
8544,2983.0,6671.0,2609.0,5212.0,2896.0,6889.0,3213.0,6842.0,2948.0,6738.0,...,2169.0,5476.0,2206.0,5760.0,2387.0,6793.0,2359.0,6918.0,2274.0,6980.0
8504,1377.0,3196.0,1281.0,2613.0,1497.0,2682.0,1380.0,3035.0,1413.0,3001.0,...,1111.0,2902.0,1127.0,2793.0,1157.0,2890.0,1243.0,2872.0,1052.0,3326.0
8528,516.0,3724.0,487.0,3219.0,626.0,2772.0,678.0,3339.0,592.0,3215.0,...,223.0,3131.0,308.0,2544.0,281.0,2725.0,311.0,2587.0,302.0,2722.0
8501,881.0,2531.0,667.0,2018.0,805.0,2649.0,973.0,3172.0,1118.0,3243.0,...,807.0,2406.0,674.0,2319.0,1092.0,2127.0,1259.0,2458.0,802.0,2717.0
8507,1676.0,1446.0,1863.0,1132.0,2058.0,1533.0,1867.0,1930.0,1733.0,1939.0,...,1888.0,2250.0,1970.0,2626.0,1684.0,2608.0,1366.0,2598.0,1462.0,3004.0
8518,213.0,1674.0,128.0,1561.0,157.0,1403.0,135.0,1545.0,153.0,1901.0,...,103.0,1249.0,65.0,1156.0,100.0,1131.0,121.0,1363.0,91.0,1553.0
8529,1168.0,983.0,871.0,915.0,1081.0,1155.0,1238.0,1118.0,1352.0,1299.0,...,3000.0,1259.0,2606.0,1129.0,2812.0,1176.0,2026.0,1035.0,1657.0,1091.0
8541,230.0,565.0,192.0,428.0,287.0,355.0,236.0,486.0,297.0,1026.0,...,142.0,1024.0,198.0,1223.0,299.0,1386.0,249.0,1842.0,237.0,2105.0


In [36]:
df_final.to_excel('전자기기.xlsx')